In [1]:
import pandas as pd
import qrcode.image.svg
from PIL import Image, ImageDraw, ImageFont
from cairosvg import svg2png

In [2]:
# read and display imported data
df = pd.read_csv("people.csv", keep_default_na=False)
# print(df)

# define template to use
template = Image.open("templates/390x600.png")
width, heigth = template.size

# define qr parameters
factory = qrcode.image.svg.SvgPathImage

In [3]:
# convert to a dictionary of records
records = df.to_dict('records')

# select a font family to write with
lato = ImageFont.truetype("fonts/Lato/Lato-Regular.ttf", size=24)
montserrat = ImageFont.truetype("fonts/Montserrat/Montserrat-Bold.ttf", size=24)

In [4]:
# defines the drawing parameters on the card
def generate_card(data, trimmed_id, card, x):

    # take a clear copy of the template
    clear = card.copy()

    # import pohoto and paste on card
    pic = Image.open(f"photos/{trimmed_id}.jpg").resize((150, 150), Image.ANTIALIAS)
    clear.paste(pic, (120, 50))

    # start drawing on card
    draw = ImageDraw.Draw(clear)

    # draw ID text
    draw.text((157, 380), str(data['id']), font=lato, fill='#7D909B')

    # draw text with PIL text anchors MA, with image x = width/2 and y = distance from top to center text horizontally
    draw.text((x/2, 238), data['name'], font=montserrat, fill='#0F5D89', anchor='ma')
    draw.text((x/2, 276), data['departament'], font=lato, fill='#7D909B', anchor='ma')
    draw.text((x/2, 313), data['position'], font=lato, fill='#7D909B', anchor='ma')

    # create QR, from record data and save as temp svg
    svg = qrcode.make(data, image_factory=factory)
    svg.save('temp/temp.svg')

    # convert temp svg to temp png
    svg2png(url="temp/temp.svg", write_to="temp/temp.png")

    # import temp png and paste onto card as a mask to avoid transparency overwriting the card
    temp = Image.open(f"temp/temp.png").resize((110, 110))
    clear.paste(temp, (265, 475), temp)

    # TO-DO find a method to create the svg and paste/draw it directly, avoiding conversions and saving/re-saving entirely

    return clear

In [5]:
# loops each record to be drawn
for record in records:
    trimmed_id = record['id'].replace('.','').replace('-','')
    card_done = generate_card(record, trimmed_id, template, width)
    card_done.save(f"cards/{trimmed_id}.png")